In [1]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import dfols
import signal
from scipy.integrate import solve_ivp
from scipy.fft import fft, fftfreq, fftshift
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy import interpolate
from stopit import threading_timeoutable as timeoutable
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from batfuns import *
plt.rcParams = set_rc_params(plt.rcParams)

eSOH_DIR = "../data/esoh/"
oCV_DIR = "../data/ocv/"
fig_DIR = "../figures/figures_model/"
res_DIR = "../data/results_model/"
%matplotlib widget

In [2]:
parameter_values = get_parameter_values()
spme = pybamm.lithium_ion.SPM(
    {
        "SEI": "ec reaction limited",
        "loss of active material": "stress-driven",
        "lithium plating": "irreversible",
        # "calculate discharge energy": "true"
    }
)
param=spme.param

In [3]:
cell = 1
cell_no,dfe,dfe_0,dfo_0,N,N_0 = load_data(cell,eSOH_DIR,oCV_DIR)
eps_n_data,eps_p_data,c_rate_c,c_rate_d,dis_set,Temp,SOC_0 = init_exp(cell_no,dfe,spme,parameter_values)
print(Temp)

25


In [4]:
parameter_values.update(
    {
      "Positive electrode LAM constant proportional term [s-1]": 1.27152e-07,
      "Negative electrode LAM constant proportional term [s-1]": 1.27272e-06,
      "Positive electrode LAM constant exponential term": 1.1992,
      "Negative electrode LAM constant exponential term": 1.1992,
      "SEI kinetic rate constant [m.s-1]":  4.60788219e-16, #1.08494281e-16 , 
      "EC diffusivity [m2.s-1]": 4.56607447e-19,#8.30909086e-19,
      "SEI growth activation energy [J.mol-1]": 1.87422275e+04,#1.58777981e+04,
      # "Exchange-current density for plating [A.m-2]":0.001,
      # "Lithium plating transfer coefficient": 0.7,
      # "Dead lithium decay constant [s-1]": 1E-4,
    },
    check_already_exists=False,
)

In [5]:
experiment = pybamm.Experiment(
    [
        ("Discharge at "+c_rate_d+dis_set,
         "Rest for 5 min",
         "Charge at "+c_rate_c+" until 4.2V", 
         "Hold at 4.2V until C/100")
    ] *dfe.N.iloc[-1],
    # ] *40,
    termination="50% capacity",
#     cccv_handling="ode",
)

In [6]:
# all_sumvars_dict = cycle_adaptive_simulation(spme, parameter_values, experiment,SOC_0, save_at_cycles=1)

In [6]:
model = spme
SOC_0=1
save_at_cycles=None

In [7]:
for var in model.initial_conditions:
    print(var.name)

Discharge capacity [A.h]
X-averaged negative electrode active material volume fraction
X-averaged positive electrode active material volume fraction
X-averaged negative particle concentration
X-averaged positive particle concentration
X-averaged outer SEI thickness
X-averaged lithium plating concentration
X-averaged dead lithium concentration


In [14]:
experiment_one_cycle = pybamm.Experiment(
        experiment.operating_conditions_cycles[:1],
        termination=experiment.termination_string,
        cccv_handling=experiment.cccv_handling,
    )
Vmin = 3.0
Vmax = 4.2
esoh_model = pybamm.lithium_ion.ElectrodeSOH()
esoh_sim = pybamm.Simulation(esoh_model, parameter_values=parameter_values)
param = model.param
esoh_solver = pybamm.lithium_ion.ElectrodeSOHSolver(parameter_values, param)
Cn = parameter_values.evaluate(param.n.cap_init)
Cp = parameter_values.evaluate(param.p.cap_init)
eps_n = parameter_values["Negative electrode active material volume fraction"]
eps_p = parameter_values["Positive electrode active material volume fraction"]
C_over_eps_n = Cn / eps_n
C_over_eps_p = Cp / eps_p
c_n_max = parameter_values.evaluate(param.n.prim.c_max)
c_p_max = parameter_values.evaluate(param.p.prim.c_max)
n_Li_init = parameter_values.evaluate(param.n_Li_particles_init)

esoh_sol = esoh_sim.solve(
    [0],
    inputs={"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li_init},
    solver=pybamm.AlgebraicSolver(),
)

parameter_values.update(
    {
        "Initial concentration in negative electrode [mol.m-3]": esoh_sol[
            "x_100"
        ].data[0]
        * c_n_max,
        "Initial concentration in positive electrode [mol.m-3]": esoh_sol[
            "y_100"
        ].data[0]
        * c_p_max,
        
    }
)

sim_ode = pybamm.Simulation(
    model, experiment=experiment_one_cycle, parameter_values=parameter_values,
    solver=pybamm.CasadiSolver("safe")
)
sol0 = sim_ode.solve(initial_soc=SOC_0)
model = sim_ode.solution.all_models[0]
cap0 = sol0.summary_variables["Capacity [A.h]"]

In [15]:
if experiment.termination == {}:
    event = None
else:

    def capacity_cutoff(t, y):
        sol = y_to_sol(y, esoh_sim, model)
        cap = pybamm.make_cycle_solution([sol], esoh_solver, True)[1]["Capacity [A.h]"]
        return cap / cap0 - experiment_one_cycle.termination["capacity"][0] / 100

    capacity_cutoff.terminal = True

num_cycles = len(experiment.operating_conditions_cycles)

In [16]:
def sol_to_y(sol, loc="end"):
    if loc == "start":
        pos = 0
    elif loc == "end":
        pos = -1
    model = sol.all_models[0]
    n_Li = sol["Total lithium in particles [mol]"].data[pos].flatten()
    Cn = sol["Negative electrode capacity [A.h]"].data[pos].flatten()
    Cp = sol["Positive electrode capacity [A.h]"].data[pos].flatten()
    # y = np.concatenate([n_Li, Cn, Cp])
    y = n_Li
    for var in model.initial_conditions:
        if var.name not in [
            "X-averaged negative particle concentration",
            "X-averaged positive particle concentration",
            "Discharge capacity [A.h]",
            "Porosity times concentration",
        ]:
            print(var.name)
            value = sol[var.name].data
            if value.ndim == 1:
                value = value[pos]
            elif value.ndim == 2:
                value = value[:, pos]
            elif value.ndim == 3:
                value = value[:, :, pos]
            y = np.concatenate([y, value.flatten()])
        elif var.name == "Porosity times concentration":
            print(var)
            for child in var.children:
                value = sol[child.name].data
                if value.ndim == 1:
                    value = value[pos]
                elif value.ndim == 2:
                    value = value[:, pos]
                elif value.ndim == 3:
                    value = value[:, :, pos]
                y = np.concatenate([y, value.flatten()])
    return y

In [17]:
sol =sol0
loc="start"
if loc == "start":
    pos = 0
elif loc == "end":
    pos = -1

In [18]:
model = sol.all_models[0]
n_Li = sol["Total lithium in particles [mol]"].data[pos].flatten()
Cn = sol["Negative electrode capacity [A.h]"].data[pos].flatten()
Cp = sol["Positive electrode capacity [A.h]"].data[pos].flatten()
# y = np.concatenate([n_Li, Cn, Cp])
y = n_Li
for var in model.initial_conditions:
    if var.name not in [
        "X-averaged negative particle concentration",
        "X-averaged positive particle concentration",
        "Discharge capacity [A.h]",
        "Porosity times concentration",
    ]:
        print(var.name)
        value = sol[var.name].data
        print(value.shape)
        if value.ndim == 1:
            value = value[pos]
        elif value.ndim == 2:
            value = np.average(value[:, pos])
        elif value.ndim == 3:
            value = np.average(value[:, :, pos])
        y = np.concatenate([y, value.flatten()])
    elif var.name == "Porosity times concentration":
        print(var)
        for child in var.children:
            print(child.name)
            value = sol[child.name].data
            print(value.shape)
            if value.ndim == 1:
                value = value[pos]
            elif value.ndim == 2:
                value = np.average(value[:, pos])
            elif value.ndim == 3:
                value = np.average(value[:, :, pos])
            y = np.concatenate([y, value.flatten()])

X-averaged negative electrode active material volume fraction
(606,)
X-averaged positive electrode active material volume fraction
(606,)
X-averaged outer SEI thickness
(606,)
X-averaged lithium plating concentration
(606,)
X-averaged dead lithium concentration
(606,)


In [19]:
y

array([0.1929887, 0.61     , 0.445    , 1.       , 0.       , 0.       ])

In [20]:
len(y)

6

In [14]:
sdfsdfsd

NameError: name 'sdfsdfsd' is not defined

In [21]:
num_cycles = len(experiment.operating_conditions_cycles)
if save_at_cycles is None:
    t_eval = np.arange(1, num_cycles + 1)
elif save_at_cycles == -1:
    t_eval = None
else:
    t_eval = np.arange(1, num_cycles + 1, save_at_cycles)
y0 = sol_to_y(sol0, loc="start")
y = y0

X-averaged negative electrode active material volume fraction
X-averaged positive electrode active material volume fraction
X-averaged outer SEI thickness
X-averaged lithium plating concentration
X-averaged dead lithium concentration


In [15]:
n_Li = y[0]
Cn = C_over_eps_n * y[1]
Cp = C_over_eps_p * y[2]

esoh_sol = esoh_sim.solve(
    [0],
    inputs={"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp, "n_Li": n_Li},
)
esoh_sim.built_model.set_initial_conditions_from(esoh_sol)
ics = {}
x_100 = esoh_sol["x_100"].data[0]
y_100 = esoh_sol["y_100"].data[0]
x_0 = esoh_sol["x_0"].data[0]
y_0 = esoh_sol["y_0"].data[0]

In [16]:
for var in model.initial_conditions:
    print(var.name)

Discharge capacity [A.h]
X-averaged negative electrode active material volume fraction
X-averaged positive electrode active material volume fraction
X-averaged negative particle concentration
X-averaged positive particle concentration
Porosity times concentration
Outer SEI thickness


In [17]:
start = 1
for var in model.initial_conditions:
    if var.name == "X-averaged negative particle concentration":
        ics[var.name] = ((x_100-x_0)*SOC_0+x_0) * np.ones((model.variables[var.name].size, 2))
    elif var.name == "X-averaged positive particle concentration":
        ics[var.name] = ((y_100-y_0)*SOC_0+y_0)  * np.ones((model.variables[var.name].size, 2))
        
    elif var.name == "Discharge capacity [A.h]":
        ics[var.name] = np.zeros(1)
    else:
        if var.name == "Porosity times concentration":
            for child in var.children:
                # end = start + model.variables[child.name].size
                # ics[child.name] = y[start:end, np.newaxis]
                end = start + 1
                ics[child.name] = y[start] * np.ones((model.variables[var.name].size, 1))
                print(child.name)
                print(ics[child.name])
                start = end
        else:
            # end = start + model.variables[var.name].size
            # ics[var.name] = y[start:end, np.newaxis]
            end = start + 1
            ics[var.name] = y[start] * np.ones((model.variables[var.name].size, 1))
            start = end
            print(var.name)
            print(ics[var.name])
    

X-averaged negative electrode active material volume fraction
[[0.61]]
X-averaged positive electrode active material volume fraction
[[0.445]]
Negative electrode porosity times concentration
[[0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]
 [0.3]]
Separator porosity times concentration
[[0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]
 [0.4]

In [ ]:
len(y)

In [18]:
for var in model.initial_conditions:
    # if var.name == "Porosity times concentration":
        print(var.name)
        var1 = var

Discharge capacity [A.h]
X-averaged negative electrode active material volume fraction
X-averaged positive electrode active material volume fraction
X-averaged negative particle concentration
X-averaged positive particle concentration
Porosity times concentration
Outer SEI thickness


In [19]:
model.variables[var1.name].domains['primary']

['negative electrode']

In [20]:
init_cond = model.initial_conditions

In [21]:
init_cond

{Variable(0x5e55c2873187ce3b, Discharge capacity [A.h], children=[], domains={}): Vector(-0x6af52df65ef574d5, Column vector of length 1, children=[], domains={}),
 Variable(0x79c80969d5bffb65, X-averaged negative electrode active material volume fraction, children=[], domains={'primary': ['current collector']}): Vector(0x5ad1a309d230bd96, Column vector of length 1, children=[], domains={'primary': ['current collector']}),
 Variable(0x6266b3f6664d09e, X-averaged positive electrode active material volume fraction, children=[], domains={'primary': ['current collector']}): Vector(-0x4eb5155eec27d50b, Column vector of length 1, children=[], domains={'primary': ['current collector']}),
 Variable(-0x66f077a4ef1e44c9, X-averaged negative particle concentration, children=[], domains={'primary': ['negative particle'], 'secondary': ['current collector']}): Vector(-0x4d3a2717c55ff3b5, Column vector of length 20, children=[], domains={'primary': ['negative particle'], 'secondary': ['current collect

In [ ]:
dfsdfs

In [ ]:
model.initial_conditions

In [22]:
 model.set_initial_conditions_from(ics)